# Description

The Fashion MNIST dataset is made up of images from fashion chain: Zalando. It contains a training set of 60,000 images, and a test set of 10,000 images. Each image is a 28 x 28 gray scale image associated with the label from 10 classes. Each training and test example is assigned to one of the following labels. Label zero refers to a T-shirt. Label one is a trouser, and so on.  

# Problem Statement

Train a neural network model so that it can distinguish between the different classes of objects. What do the different images in the dataset look like? 


In [ ]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import numpy as np
np.random.seed(0)

In [ ]:
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn

mean, std = (0.5,), (0.5,)

# Create a transform and normalise data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean, std)
                              ])

# Download FMNIST training dataset and load training data
trainset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download FMNIST test dataset and load test data
testset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
# 
class FMNIST(nn.Module): # FMNIST is a derived class or subclass of nn.Module base class
  def __init__(self): # define class method with first parameter as self
    super().__init__() # make a call to the nn.Module's dunder init method
    self.fc1 = nn.Linear(784, 128) # we start off with 784 input nodes and must be 784 because these are the number of pixels in the input image.
    self.fc2 = nn.Linear(128,64) # We then have two hidden layers with 128 nodes and 64 nodes chosen arbitrarily 
    self.fc3 = nn.Linear(64,10) # The final output layer has 10 nodes because there are 10 classes in the dataset
    
  def forward(self, x): # In forward method, we are defining the exact order of the different layers
    x = x.view(x.shape[0], -1)
    
    x = F.relu(self.fc1(x)) # ReLu is a non-linear activation function that allows the nodes to learn more complex structures in an image.

    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    x = F.log_softmax(x, dim=1) # to determine the actual probability for each of the individual classes 
   
    
    return x

model = FMNIST()

In [ ]:
from torch import optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 3 # we run only 3 epochs or runs, through the training data

for i in range(num_epochs):
    cum_loss = 0

    for images, labels in trainloader: # take a batch of images and targets from trainloader
        optimizer.zero_grad() # zero out gradients at the start of each epoch
        output = model(images) # forward pass: run this batch through neural network to see what predictions neural network provides
        # calculate the loss of the neural network for that batch
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() # update weights of the neural network
        
        cum_loss += loss.item()
     
    print(f"Training loss: {cum_loss/len(trainloader)}") 

In [ ]:
# At this point, we have trained our neural network.
# Let's see how well it does on images that it hasn't seen before.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

images, labels = next(iter(testloader))

# You now have a choice of any 64 images to try this on. Pick any number between zero and 63 inclusive 
# because the batch of test images is 64, so pick number less than 64 and enter that number for the test image ID below. 
test_image_id = 52
img = images[test_image_id].view(1, 784) 

with torch.no_grad():
    logps = model(img)

In [ ]:
ps = torch.exp(logps)
nps = ps.numpy()[0]
FMNIST_labels = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sport Shoes','Bag','Ankle Boot']
plt.xticks(np.arange(10),labels=FMNIST_labels,rotation='vertical')
plt.bar(np.arange(10), nps)

In [ ]:
# for test_image_id = 52 it's predicting a sandal. Let's confirm if that's the case in the next cell block.

In [ ]:
def denormalize(tensor):
  tensor = tensor*0.5 + 0.5
  return tensor
  
img = img.view(28,-1)
img = denormalize(img)
plt.imshow(img,cmap='gray')

In [ ]:
# I would agree that this does look like a sandal.